# Blah

In [49]:
from IPython.display import display, HTML, Image , Markdown
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F
import os ,configparser ,json

import ipyregulartable as rt
import pandas as pd
import numpy as np

display(Markdown("### Initialization"))

# Source various helper functions
%run ../scripts/notebook_helpers.py

PROJECT_HOME_DIR = '../../'
config = get_config(PROJECT_HOME_DIR)
sp_session = connect_to_snowflake(PROJECT_HOME_DIR)

if(sp_session == None):
    raise Exception(f'Unable to connect to snowflake. Validate connection information ')

sp_session.sql(f''' use role {config['APP']['role']}; ''').collect()
sp_session.sql(f''' use database {config['APP']['database']}; ''').collect()
sp_session.sql(f''' use schema  {config['APP']['database']}.{config['APP']['schema']}; ''').collect()
sp_session.sql(f''' use warehouse {config['APP']['warehouse']}; ''').collect()

df = sp_session.sql('select current_user() ,current_role() ,current_database() ,current_schema();').to_pandas()
display(df)

### Initialization

,CURRENT_USER(),CURRENT_ROLE(),CURRENT_DATABASE(),CURRENT_SCHEMA()
0,VSEKAR,DEV_PCTRANSPERANCY_DEMO_RL,SFLK_PRICING_TRANSPERANCY,PUBLIC


In [50]:
from datetime import datetime
from datetime import timedelta
import time
import hashlib

def get_basename_of_datafile(p_datafile:str) -> str:
    base = os.path.basename(p_datafile)
    base = base.replace('-','_')
    fl_base = os.path.splitext(base)
    
    return fl_base[0]

data_file = '2022_10_01_priority_health_HMO_in-network-rates.zip'
stage_path = 'data_stg/price_transperancy'

data_file = '2022_07_01_priority_health_HMO_in-network-rates.json'
# data_file = '2022-11-01_cigna-health-life-insurance-company_national-oap_in-network-rates.json.gz'
stage_path = 'ext_data_stg/data'

datafl_basename = get_basename_of_datafile(data_file)
datafl_basename

'2022_07_01_priority_health_HMO_in_network_rates'

In [51]:
# Upload libraries to stage
lib_stage = config['APP']['lib_stage']

upload_locallibraries_to_p_stage(sp_session ,'../python/in_network_udtf' ,config['APP']['database'] ,'public' ,lib_stage ,'scripts')

stmts = [
    f''' 
    create or replace procedure innetwork_rates_nr_dagbuilder(
            batch_size integer ,stage_path varchar ,staged_data_flname varchar ,target_stage_for_segment_files varchar
            ,parallels number ,execution_warehouse varchar ,rerun boolean)
        returns variant
        language python
        runtime_version = '3.8'
        packages = ('snowflake-snowpark-python' ,'pandas', 'ijson')
        imports = ('@sflk_pricing_transperancy.public.lib_stg/scripts/negotiation_arrangements_dagbuilder.py')
        handler = 'negotiation_arrangements_dagbuilder.main'
        ;
    '''
]
for stmt in stmts:
    sp_session.sql(stmt).collect()


 Uploading library to stage: sflk_pricing_transperancy.public.lib_stg 
    ../python/in_network_udtf/negotiation_arrangements_dagbuilder.py => @lib_stg/scripts
    ../python/in_network_udtf/negotiation_arrangements.py => @lib_stg/scripts
    ../python/in_network_udtf/negotiation_arrangements_dagbuilder_tmp.py => @lib_stg/scripts


In [52]:
# Dag cleanup (blank)

# sp_session.sql(f''' use warehouse DEMO_BUILD_WH; ''').collect()

sp_session.sql(f''' SHOW TASKS IN  DATABASE {config['APP']['database']}; ''').collect()

df = sp_session.sql(f''' 
   SELECT 
      concat('drop task if exists ', "name" ,';') as drop_stmt
   FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()))
   --where "state" = 'suspended'
   ORDER BY "state" asc;
''').to_pandas()

for index, row in df.iterrows():
   sp_session.sql(row['DROP_STMT']).collect()

# display(df)
# rt.RegularTableWidget(df)

In [53]:
# Dag & task build up

start_time = time.time()
print(f'Started at: {datetime.now().strftime("%H:%M:%S")}')

sql_stmts = [
    f''' alter warehouse dev_pctransperancy_demo_wh set warehouse_size = XSMALL; '''
    ,f'''truncate table {config['APP']['database']}.public.segment_task_execution_status; '''
    ,f''' call {config['APP']['database']}.public.innetwork_rates_nr_dagbuilder(
        5 ,'{stage_path}' ,'{data_file}' ,'ext_data_stg/data_pricing_parsed'
        ,10 ,current_warehouse() ,true
        );'''
]
for stmt in sql_stmts:
    print(stmt)
    sp_session.sql(stmt).collect()

end_time = time.time()
print(f'Ended at: {datetime.now().strftime("%H:%M:%S")}')

elapsed_time = end_time - start_time
elapsed = str(timedelta(seconds=elapsed_time))
print(f'Elapsed: {elapsed}')

Started at: 12:10:30
 alter warehouse dev_pctransperancy_demo_wh set warehouse_size = XSMALL; 
truncate table sflk_pricing_transperancy.public.segment_task_execution_status; 
 call sflk_pricing_transperancy.public.innetwork_rates_nr_dagbuilder(
        5 ,'ext_data_stg/data' ,'2022_07_01_priority_health_HMO_in-network-rates.json' ,'ext_data_stg/data_pricing_parsed'
        ,10 ,current_warehouse() ,true
        );


In [ ]:
df = sp_session.table(f'''{config['APP']['database']}.public.task_to_segmentids''').to_pandas()
# display(df)
rt.RegularTableWidget(df)

In [ ]:
# segment parsing

start_time = time.time()
print(f'Started at: {datetime.now().strftime("%H:%M:%S")}')
dag_start_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f'DAG Start time: {dag_start_time} ')
# XSMALL | SMALL | MEDIUM | LARGE | XLARGE | XXLARGE | XXXLARGE | X4LARGE | X5LARGE | X6LARGE
sql_stmts = [
    f''' alter warehouse dev_pctransperancy_demo_wh set max_concurrency_level = 1 '''
    ,f''' alter warehouse dev_pctransperancy_demo_wh set warehouse_size = XXXLARGE; '''
    ,f'''truncate table {config['APP']['database']}.public.segment_task_execution_status; '''
    ,f''' execute task DAG_ROOT_{datafl_basename}; '''
]
for stmt in sql_stmts:
    sp_session.sql(stmt).collect()

end_time = time.time()
print(f'Ended at: {datetime.now().strftime("%H:%M:%S")}')

elapsed_time = end_time - start_time
elapsed = str(timedelta(seconds=elapsed_time))
print(f'Elapsed: {elapsed}')



---

Observations

In [ ]:
import pandas as pd

# Monitor task to ensure that there are no errors

# dag_start_time = '2022-12-04 07:58:00'
print(f'DAG start time: {dag_start_time}')

sql_stmt = f'''
select 
    name ,state ,error_code ,error_message
  from table(information_schema.task_history(
    scheduled_time_range_start=>to_timestamp_ltz('{dag_start_time} -0500')
   -- ,ERROR_ONLY => TRUE
  ))
 -- where state not in ('SUCCEEDED')
  order by state asc
;'''
df = sp_session.sql(sql_stmt).to_pandas()

#display(df)
rt.RegularTableWidget(df)


In [ ]:
df = sp_session.table(f'''{config['APP']['database']}.public.segment_task_execution_status''').to_pandas()
rt.RegularTableWidget(df)

In [ ]:
sql_stmt = f'''
select min(l.inserted_at) as started_at 
    ,max(l.inserted_at) as latest 
    ,TIMESTAMPDIFF(minutes, started_at ,latest) as elapsed_min_latest
    ,TIMESTAMPDIFF(minutes, started_at ,current_timestamp()) as elapsed_min 
    ,least(elapsed_min_latest ,elapsed_min)/60 as elapsed_hr
    ,count(l.task_name) as task_count
    ,timediff(minutes, started_at ,latest) as elapsed_minutes 
from {config['APP']['database']}.public.segment_task_execution_status as l
 ;
    '''
df = sp_session.sql(sql_stmt).to_pandas()
display(df)
# rt.RegularTableWidget(df)

In [ ]:
aSasaSasas

In [ ]:


sql_stmt = f'''
        select 'total_headers' ,sum(negotiated_rates_count)
            from {config['APP']['database']}.public.in_network_rates_segment_header_V2
        union
        select 'current_records_loaded' ,count(*) as rec_count 
            from {config['APP']['database']}.public.negotiated_arrangment_segments_v2
            group by data_file
        ;
    '''
df = sp_session.sql(sql_stmt).to_pandas()
display(df)

In [ ]:
sql_stmt = f'''
        select segment_id ,count(*) as rec_count 
        from {config['APP']['database']}.public.negotiated_arrangment_segments_v2
        group by segment_id
        ;
    '''
df = sp_session.sql(sql_stmt).to_pandas()
display(df)


In [ ]:
sql_stmt = f'''
        select *
        from {config['APP']['database']}.public.segment_task_execution_status
        ;
    '''
df = sp_session.sql(sql_stmt).to_pandas()
display(df)

In [ ]:
# x_df = df.set_index('SEGMENT_ID')
y_df = pd.qcut(df['NEGOTIATED_RATES_COUNT'], 10 ,labels=False)
# q_df.columns = 'CLASS'
# q_df

q_df = pd.DataFrame({'CLASS':y_df.values})
q_df
# q_df.value_counts()

In [ ]:
# df.join(q_df)
# q_df.columns
# df.columns
x_df = pd.concat([df ,q_df] ,axis=1)
x_df

In [ ]:
x_df.groupby('CLASS')['NEGOTIATED_RATES_COUNT'].sum()

In [ ]:
# from datetime import datetime
# now = datetime.now()

# start_time = now.strftime("%H:%M:%S")
# print(f'Started at: {start_time}')

# sql_stmts = [
#     f''' alter warehouse dev_pctransperancy_demo_wh set warehouse_size = XSMALL; '''
#     ,f'''truncate table {config['APP']['database']}.public.in_network_rates_segment_header_V2; '''
#     ,f''' call {config['APP']['database']}.public.innetwork_rates_segheader(
#             1000 ,'data_stg/price_transperancy' ,'2022_10_01_priority_health_HMO_in-network-rates.zip');'''
# ]
# for stmt in sql_stmts:
#     sp_session.sql(stmt).collect()

# now = datetime.now()
# end_time = now.strftime("%H:%M:%S")
# print(f'Ended at: {end_time}')

---
### TBL

In [ ]:
df = sp_session.table(f'''{config['APP']['database']}.public.in_network_rates_segment_header_V2''').limit(10).to_pandas()
display(df)

#### TODO : Perform some model analysis 
 - accuracy score
 - error rate calculation
 - confusion matrix
  ...

--- 
### Closeout

    With that we are finished this section of the demo setup

In [ ]:
# sp_session.close()
print('Finished!!!')

In [ ]:
employees = [('Stuti', 28, 'Varanasi', 20000),
            ('Saumya', 32, 'Delhi', 25000),
            ('Aaditya', 25, 'Mumbai', 40000),
            ('Saumya', 32, 'Delhi', 35000),
            ('Saumya', 32, 'Delhi', 30000),
            ('Saumya', 32, 'Mumbai', 20000),
            ('Aaditya', 40, 'Dehradun', 24000),
            ('Seema', 32, 'Delhi', 70000)
            ]
 
# Create a DataFrame object from list
df = pd.DataFrame(employees,
                columns =['Name', 'Age',
                        'City', 'Salary'])
# Show the dataframe
df

In [ ]:
df.iloc[3:5,]